In [ ]:
%load_ext autoreload
%autoreload 2
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

from pymongo import MongoClient

from azuresigmf import mongo_tools
from azuresigmf.sigmf_db_collection import SigMFDBCollection
from sigmf import SigMFFile

# for pretty pictures
import matplotlib as mpl
import scipy
import numpy as np

# plot snapshots of the spectrum
def plot(samps, Fs, Fc):
    spectrum, freqs, t, im= mpl.pyplot.specgram(samps, NFFT=256, Fs=Fs, Fc=Fc)
    mpl.pyplot.imshow(10*np.log10(np.abs(spectrum)).T, aspect="auto", extent=[freqs[0]/1e6,freqs[-1]/1e6,t[0]*1e6,t[-1]*1e6])
    ax = mpl.pyplot.gca()
    _= ax.set_ylabel("Time (us)")
    _= ax.set_xlabel("Frequency (MHz)")
    mpl.pyplot.tight_layout()

In [ ]:
# Acquire a credential object
credential = DefaultAzureCredential()

# get a Key Vault client to access secrets 
kv_uri = "https://sigmf-db-kv.vault.azure.net"
kv_client = SecretClient(vault_url=kv_uri, credential=credential)

In [ ]:
# setup constants
mongo_conn_str = kv_client.get_secret("mongo-conn-str").value
mongo_db_name = "sigmf-meta"
collection_name = "recordings"

# Set up a connection to a Mongo database using the connection string from our keyvault

In [ ]:
# connect to the database
dbclient = MongoClient(mongo_conn_str)
collection = dbclient[mongo_db_name][collection_name]

# set up our wrapper
sigmf_collection = SigMFDBCollection(collection, credential)

# Find a SigMF file that includes at least one LTE signal detection

In [ ]:
# Find the first SigMF file that include at least one "LTE" annotation
signal = sigmf_collection.find_one( {"annotations": {"$elemMatch": { "core:description": "LTE"}}})

# Read the recorded samples from that file

In [ ]:
# get the samples from this signal
print(f"reading in {signal.sample_count/1e6} million samples")
samples = signal.read_samples(0, signal.sample_count)

In [ ]:
# plot
%matplotlib widget

sample_rate = signal.get_global_field(SigMFFile.SAMPLE_RATE_KEY)
capture = signal.get_capture_info(0)
freq_center = capture.get(SigMFFile.FREQUENCY_KEY, 0)

plot(samples, Fs=sample_rate, Fc=freq_center)

# Now lets find all the DECT6 detections in all of our SigMF Files

In [ ]:
# Match all documents that have at least one "DECT6" annotation
match_stage = {
    "$match": {
        "annotations": {
            "$elemMatch": { "core:description": "DECT6"}
        }
    }
}

# and filter down to only the DECT6 annotations in each document
filter_spec = { 
    "$filter": {
        "input": "$annotations", 
        "as":"out", 
        "cond": {"$eq":["$$out.core:description", "DECT6"]}
    }
}

project_stage = { 
    "$project": {
        "uri":1,
        "captures": 1,
        "global.core:datatype": 1,
        "annotations": filter_spec 
    }
}

results = sigmf_collection.aggregate(
    [
       match_stage,
       project_stage 
    ] 
)

# Get the first detection from the first file

In [ ]:
# select the first annotation from the first result and get the samples from it
signal = results[0]
annotation = signal.get_annotations()[0]

samples = signal.read_annotation_samples(annotation)

In [ ]:
# plot
%matplotlib widget

sample_rate = signal.get_global_field(SigMFFile.SAMPLE_RATE_KEY)
capture = signal.get_capture_info(0)
freq_center = capture.get(SigMFFile.FREQUENCY_KEY, 0)

plot(samples, Fs=sample_rate, Fc=freq_center)
